# **09 - Memory and Conversations: Building Stateful AI Applications**

## **Overview**
In this notebook, we'll learn how to add memory to LLM applications, enabling them to maintain context across interactions. You'll build chatbots that remember previous conversations, track entities, and maintain long-term knowledge.

## **Learning Objectives**
By the end of this notebook, you will be able to:
- Implement different types of conversation memory
- Build chatbots with context awareness
- Use entity memory to track specific information
- Create summary-based memory for long conversations
- Implement knowledge graph memory
- Design persistent memory systems

## **Prerequisites**
- Completion of notebooks 01-08
- Understanding of chains and agents
- Basic knowledge of conversation design

## **Back-and-Forth Teaching Pattern**
This notebook follows our pattern:
1. **Instructor Activity**: Demonstrates a concept with complete examples
2. **Learner Activity**: You apply the concept with guidance and hidden solutions

## **Setup**

Let's install and import the necessary libraries:

In [ ]:
# Install required packages
!pip install langchain langchain-community langchain-openai redis faiss-cpu

In [ ]:
import os
from typing import List, Dict, Any, Optional
from langchain_openai import ChatOpenAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    ConversationBufferWindowMemory,
    ConversationEntityMemory,
    ConversationKGMemory,
    CombinedMemory,
    VectorStoreRetrieverMemory
)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import ConversationChain, LLMChain
from langchain.schema import HumanMessage, AIMessage, BaseMessage
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

---

## **Instructor Activity 1: Basic Conversation Memory Types**

Let's explore different types of memory for conversations:

In [ ]:
# 1. Conversation Buffer Memory - Stores all messages
llm = ChatOpenAI(temperature=0.7)

# Create buffer memory
buffer_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create conversation chain
conversation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant having a conversation with a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

conversation = ConversationChain(
    llm=llm,
    memory=buffer_memory,
    prompt=conversation_prompt,
    verbose=True
)

# Have a conversation
print("Buffer Memory Conversation:\n")
response1 = conversation.predict(input="Hi! My name is Alice and I love programming.")
print(f"AI: {response1}\n")

response2 = conversation.predict(input="What's my name?")
print(f"AI: {response2}\n")

response3 = conversation.predict(input="What did I tell you I love?")
print(f"AI: {response3}\n")

# Check memory contents
print("\nMemory Contents:")
print(buffer_memory.chat_memory.messages)

In [ ]:
# 2. Conversation Buffer Window Memory - Keeps only last K messages

window_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    k=2  # Keep only last 2 exchanges
)

window_conversation = ConversationChain(
    llm=llm,
    memory=window_memory,
    prompt=conversation_prompt,
    verbose=False
)

print("Window Memory Conversation (k=2):\n")

# Have a longer conversation
messages = [
    "Hi! I'm Bob and I'm from New York.",
    "I work as a data scientist.",
    "My favorite color is blue.",
    "What's my name?",  # Should remember (within window)
    "Where am I from?"   # Might not remember (outside window)
]

for msg in messages:
    print(f"Human: {msg}")
    response = window_conversation.predict(input=msg)
    print(f"AI: {response}\n")

print("\nWindow Memory Contents (only last 2 exchanges):")
print(window_memory.chat_memory.messages)

In [ ]:
# 3. Conversation Summary Memory - Stores summarized conversation

summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    return_messages=False
)

summary_prompt = ChatPromptTemplate.from_template(
    """The following is a friendly conversation between a human and an AI.
    The AI is talkative and provides lots of specific details from its context.
    
    Current conversation summary:
    {chat_history}
    
    Human: {input}
    AI:"""
)

summary_conversation = ConversationChain(
    llm=llm,
    memory=summary_memory,
    prompt=summary_prompt,
    verbose=False
)

print("Summary Memory Conversation:\n")

# Have a detailed conversation
detailed_messages = [
    "I'm planning a trip to Japan next month. I'm really excited about it!",
    "I want to visit Tokyo, Kyoto, and Osaka. Any recommendations?",
    "I'm particularly interested in temples, technology, and food.",
    "What should I know about Japanese etiquette?"
]

for msg in detailed_messages:
    print(f"Human: {msg}")
    response = summary_conversation.predict(input=msg)
    print(f"AI: {response[:200]}...\n")  # Truncate for display

print("\nSummary Memory Contents:")
print(f"Summary: {summary_memory.buffer}")

In [ ]:
# 4. Conversation Entity Memory - Tracks entities mentioned

entity_memory = ConversationEntityMemory(
    llm=llm,
    memory_key="entities",
    return_messages=True
)

entity_prompt = ChatPromptTemplate.from_template(
    """You are having a conversation with a human.
    You remember specific facts about entities mentioned.
    
    Relevant entity information:
    {entities}
    
    Conversation history:
    {history}
    
    Human: {input}
    AI:"""
)

# Manual chain with entity memory
def entity_conversation(human_input: str) -> str:
    # Get entity information
    entities_info = entity_memory.load_memory_variables({"input": human_input})
    
    # Get conversation history
    history = entity_memory.chat_memory.messages
    
    # Format and get response
    formatted_prompt = entity_prompt.format(
        entities=entities_info.get("entities", ""),
        history=history,
        input=human_input
    )
    
    response = llm.predict(formatted_prompt)
    
    # Save to memory
    entity_memory.save_context({"input": human_input}, {"output": response})
    
    return response

print("Entity Memory Conversation:\n")

# Conversation with multiple entities
entity_messages = [
    "I'm Sarah and I work at Google as a software engineer.",
    "My colleague John works in the marketing department at Microsoft.",
    "Tell me what you know about Sarah.",
    "What about John?"
]

for msg in entity_messages:
    print(f"Human: {msg}")
    response = entity_conversation(msg)
    print(f"AI: {response}\n")

print("\nEntity Store:")
print(entity_memory.entity_store.store)

In [ ]:
# 5. Combined Memory - Use multiple memory types together

from langchain.memory import CombinedMemory

# Create individual memories
conv_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    input_key="input",
    k=3
)

summary_memory_2 = ConversationSummaryMemory(
    llm=llm,
    memory_key="summary",
    input_key="input"
)

# Combine memories
combined_memory = CombinedMemory(memories=[conv_memory, summary_memory_2])

combined_prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant with access to both recent messages and conversation summary.
    
    Overall Summary:
    {summary}
    
    Recent Messages:
    {chat_history}
    
    Human: {input}
    AI:"""
)

combined_chain = LLMChain(
    llm=llm,
    prompt=combined_prompt,
    memory=combined_memory,
    verbose=False
)

print("Combined Memory Conversation:\n")

combined_messages = [
    "I'm learning about machine learning and deep learning.",
    "I started with linear regression and decision trees.",
    "Now I'm studying neural networks and transformers.",
    "What topics have I covered so far?"
]

for msg in combined_messages:
    print(f"Human: {msg}")
    response = combined_chain.predict(input=msg)
    print(f"AI: {response}\n")

---

## **Learner Activity 1: Build a Personal Assistant with Memory**

Create a personal assistant that remembers user preferences, tasks, and important information.

**Task**: Build an assistant that:
1. Remembers user preferences (name, interests, goals)
2. Tracks tasks and reminders across conversations
3. Maintains a summary of long-term interactions
4. Can recall specific information when asked
5. Adapts responses based on user history

Requirements:
- Use at least 2 different memory types
- Implement preference tracking
- Add task management with memory
- Handle context switching gracefully

In [ ]:
# Build your personal assistant with memory

class PersonalAssistant:
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.7)
        # TODO: Initialize different memory types
        # - Buffer memory for recent conversation
        # - Entity memory for user preferences
        # - Custom memory for tasks
        
        # Your initialization here
        pass
    
    def remember_preference(self, preference: str, value: str):
        """Store user preferences."""
        # TODO: Implement preference storage
        pass
    
    def add_task(self, task: str, due_date: Optional[str] = None):
        """Add a task to memory."""
        # TODO: Implement task storage
        pass
    
    def get_tasks(self) -> List[str]:
        """Retrieve all tasks."""
        # TODO: Implement task retrieval
        pass
    
    def chat(self, user_input: str) -> str:
        """Main conversation interface."""
        # TODO: Implement chat with memory
        # - Check for task-related requests
        # - Use appropriate memory
        # - Generate contextual response
        pass

# TODO: Test your assistant
# assistant = PersonalAssistant()
# Test conversations with preferences, tasks, and memory recall

# Your test code here

In [ ]:
# Solution (hidden by default)

"""
class PersonalAssistant:
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.7)
        
        # Recent conversation memory
        self.conversation_memory = ConversationBufferWindowMemory(
            memory_key="recent_chat",
            return_messages=True,
            k=5
        )
        
        # Entity memory for user info
        self.entity_memory = ConversationEntityMemory(
            llm=self.llm,
            memory_key="entities",
            return_messages=False
        )
        
        # Summary memory for long-term
        self.summary_memory = ConversationSummaryMemory(
            llm=self.llm,
            memory_key="long_term_summary",
            return_messages=False
        )
        
        # Custom storage for preferences and tasks
        self.preferences = {}
        self.tasks = []
        self.interaction_count = 0
        
        # Create main prompt
        self.prompt = ChatPromptTemplate.from_template(
            '''You are a personal assistant who remembers everything about the user.
            
            User Preferences:
            {preferences}
            
            Current Tasks:
            {tasks}
            
            Known Entities:
            {entities}
            
            Long-term Summary:
            {long_term_summary}
            
            Recent Conversation:
            {recent_chat}
            
            Human: {input}
            
            Assistant (be personal and reference their history when relevant):'''
        )
    
    def remember_preference(self, category: str, value: str):
        '''Store user preferences.'''
        if category not in self.preferences:
            self.preferences[category] = []
        self.preferences[category].append(value)
        
        # Also save to entity memory
        self.entity_memory.save_context(
            {"input": f"User preference - {category}: {value}"},
            {"output": f"Noted your {category} preference: {value}"}
        )
        
        return f"Preference saved: {category} = {value}"
    
    def add_task(self, task: str, due_date: Optional[str] = None):
        '''Add a task to memory.'''
        task_obj = {
            "task": task,
            "due_date": due_date or "No deadline",
            "created": datetime.now().isoformat(),
            "status": "pending"
        }
        self.tasks.append(task_obj)
        return f"Task added: {task} (Due: {due_date or 'No deadline'})"
    
    def complete_task(self, task_index: int):
        '''Mark a task as complete.'''
        if 0 <= task_index < len(self.tasks):
            self.tasks[task_index]["status"] = "completed"
            return f"Task completed: {self.tasks[task_index]['task']}"
        return "Invalid task index"
    
    def get_tasks(self, status: str = "pending") -> List[Dict]:
        '''Retrieve tasks by status.'''
        return [t for t in self.tasks if t["status"] == status]
    
    def _format_preferences(self) -> str:
        '''Format preferences for prompt.'''
        if not self.preferences:
            return "No preferences stored yet"
        
        formatted = []
        for category, values in self.preferences.items():
            formatted.append(f"{category}: {', '.join(values)}")
        return "\n".join(formatted)
    
    def _format_tasks(self) -> str:
        '''Format tasks for prompt.'''
        pending = self.get_tasks("pending")
        if not pending:
            return "No pending tasks"
        
        formatted = []
        for i, task in enumerate(pending):
            formatted.append(f"{i+1}. {task['task']} (Due: {task['due_date']})")
        return "\n".join(formatted)
    
    def chat(self, user_input: str) -> str:
        '''Main conversation interface.'''
        self.interaction_count += 1
        
        # Check for task commands
        if "add task" in user_input.lower():
            # Extract task from input
            task_text = user_input.replace("add task", "").replace("Add task", "").strip()
            self.add_task(task_text)
            user_input += " [Task added to memory]"
        
        # Check for preference setting
        if "my favorite" in user_input.lower() or "i prefer" in user_input.lower():
            # Simple extraction (in production, use NLP)
            if "favorite" in user_input:
                parts = user_input.split("favorite")
                if len(parts) > 1:
                    pref_text = parts[1].strip()
                    # Extract first few words as preference
                    words = pref_text.split()[:3]
                    if words:
                        category = words[0]
                        value = ' '.join(words[1:]) if len(words) > 1 else "noted"
                        self.remember_preference(category, value)
        
        # Load all memory variables
        recent_chat = self.conversation_memory.load_memory_variables({})
        entities = self.entity_memory.load_memory_variables({"input": user_input})
        summary = self.summary_memory.load_memory_variables({})
        
        # Format prompt
        formatted_prompt = self.prompt.format(
            preferences=self._format_preferences(),
            tasks=self._format_tasks(),
            entities=entities.get("entities", ""),
            long_term_summary=summary.get("long_term_summary", ""),
            recent_chat=recent_chat.get("recent_chat", []),
            input=user_input
        )
        
        # Get response
        response = self.llm.predict(formatted_prompt)
        
        # Save to memories
        context = {"input": user_input}
        output_dict = {"output": response}
        
        self.conversation_memory.save_context(context, output_dict)
        self.entity_memory.save_context(context, output_dict)
        self.summary_memory.save_context(context, output_dict)
        
        return response

# Test the personal assistant
print("Personal Assistant with Memory Demo\n")
print("="*50)

assistant = PersonalAssistant()

# Test conversation
test_messages = [
    "Hi! I'm Alex and I love hiking and photography.",
    "My favorite color is green and I'm learning Python.",
    "Add task: Complete the machine learning course by next Friday",
    "Add task: Buy a new camera lens",
    "What do you remember about me?",
    "What tasks do I have?",
    "I just finished my Python project!",
    "Can you remind me of my interests?"
]

for msg in test_messages:
    print(f"\nUser: {msg}")
    response = assistant.chat(msg)
    print(f"Assistant: {response}")

print("\n" + "="*50)
print("\nStored Preferences:")
print(assistant.preferences)

print("\nPending Tasks:")
for task in assistant.get_tasks():
    print(f"- {task['task']} (Due: {task['due_date']})")
"""

print("Create a personal assistant with comprehensive memory!")
print("The solution combines multiple memory types for a rich conversational experience.")

---

## **Instructor Activity 2: Advanced Memory Patterns**

Let's explore more sophisticated memory patterns:

In [ ]:
# Vector Store Memory - Semantic search over conversation history

# Create embeddings and vector store
embeddings = OpenAIEmbeddings()

# Create an empty vector store
vectorstore = FAISS.from_texts(
    ["Initial memory"],  # Need at least one item to initialize
    embeddings
)

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Create vector store memory
vector_memory = VectorStoreRetrieverMemory(
    retriever=retriever,
    memory_key="relevant_history",
    return_docs=True
)

# Custom chain with vector memory
class SemanticMemoryChat:
    def __init__(self, vector_memory, llm):
        self.vector_memory = vector_memory
        self.llm = llm
        self.all_messages = []  # Keep all messages for adding to vector store
    
    def chat(self, user_input: str) -> str:
        # Search for relevant history
        relevant_docs = self.vector_memory.retriever.get_relevant_documents(user_input)
        
        # Format relevant history
        relevant_history = "\n".join([doc.page_content for doc in relevant_docs])
        
        # Create prompt with relevant history
        prompt = f"""You are a helpful assistant. Use relevant conversation history to provide context.
        
        Relevant History:
        {relevant_history}
        
        Current Input: {user_input}
        
        Response:"""
        
        # Get response
        response = self.llm.predict(prompt)
        
        # Save to vector store
        exchange = f"Human: {user_input}\nAI: {response}"
        self.all_messages.append(exchange)
        
        # Add to vector store (in production, batch this)
        self.vector_memory.retriever.vectorstore.add_texts([exchange])
        
        return response

# Test semantic memory
semantic_chat = SemanticMemoryChat(vector_memory, llm)

print("Semantic Memory Chat:\n")

# Have a conversation with various topics
semantic_messages = [
    "I'm interested in learning about space exploration.",
    "Tell me about the Mars rovers.",
    "Let's switch topics. What's your take on artificial intelligence?",
    "How does machine learning work?",
    "Going back to our earlier topic, what's the future of space exploration?"  # Should recall space context
]

for msg in semantic_messages:
    print(f"Human: {msg}")
    response = semantic_chat.chat(msg)
    print(f"AI: {response[:200]}...\n")

In [ ]:
# Persistent Memory with JSON storage

import json
import os
from typing import Dict, List, Any

class PersistentMemory:
    """Memory that persists across sessions."""
    
    def __init__(self, file_path: str = "conversation_memory.json"):
        self.file_path = file_path
        self.memory = self._load_memory()
        self.llm = ChatOpenAI(temperature=0.7)
    
    def _load_memory(self) -> Dict:
        """Load memory from file."""
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r') as f:
                return json.load(f)
        return {
            "conversations": [],
            "user_profile": {},
            "topics_discussed": [],
            "important_facts": [],
            "session_count": 0
        }
    
    def _save_memory(self):
        """Save memory to file."""
        with open(self.file_path, 'w') as f:
            json.dump(self.memory, f, indent=2)
    
    def start_session(self):
        """Start a new conversation session."""
        self.memory["session_count"] += 1
        self.current_session = []
        return f"Session {self.memory['session_count']} started. Welcome back!"
    
    def update_profile(self, key: str, value: Any):
        """Update user profile information."""
        self.memory["user_profile"][key] = value
        self._save_memory()
    
    def add_important_fact(self, fact: str):
        """Mark something as important to remember."""
        fact_entry = {
            "fact": fact,
            "timestamp": datetime.now().isoformat(),
            "session": self.memory["session_count"]
        }
        self.memory["important_facts"].append(fact_entry)
        self._save_memory()
    
    def chat(self, user_input: str) -> str:
        """Chat with persistent memory."""
        # Check for profile updates
        if "my name is" in user_input.lower():
            # Extract name (simple approach)
            name = user_input.split("my name is")[-1].strip().split()[0]
            self.update_profile("name", name)
        
        # Check for important facts
        if "remember this" in user_input.lower() or "important:" in user_input.lower():
            self.add_important_fact(user_input)
        
        # Build context from persistent memory
        profile_str = json.dumps(self.memory["user_profile"], indent=2)
        recent_convos = self.memory["conversations"][-5:] if self.memory["conversations"] else []
        important_facts = [f["fact"] for f in self.memory["important_facts"][-3:]]
        
        prompt = f"""You are a helpful assistant with persistent memory across sessions.
        
        User Profile:
        {profile_str}
        
        Important Facts to Remember:
        {json.dumps(important_facts, indent=2)}
        
        Session Number: {self.memory['session_count']}
        
        Current Conversation:
        Human: {user_input}
        
        Assistant (reference their profile and history when relevant):"""
        
        response = self.llm.predict(prompt)
        
        # Save conversation
        convo_entry = {
            "session": self.memory["session_count"],
            "timestamp": datetime.now().isoformat(),
            "human": user_input,
            "ai": response
        }
        self.memory["conversations"].append(convo_entry)
        
        # Keep only last 100 conversations to manage file size
        if len(self.memory["conversations"]) > 100:
            self.memory["conversations"] = self.memory["conversations"][-100:]
        
        self._save_memory()
        return response
    
    def get_summary(self) -> str:
        """Get a summary of stored memory."""
        return f"""Memory Summary:
        - Total sessions: {self.memory['session_count']}
        - User profile entries: {len(self.memory['user_profile'])}
        - Conversations stored: {len(self.memory['conversations'])}
        - Important facts: {len(self.memory['important_facts'])}
        """

# Test persistent memory
persistent_chat = PersistentMemory("test_memory.json")

print("Persistent Memory Chat:\n")
print(persistent_chat.start_session())
print()

# Simulate conversation
persistent_messages = [
    "Hi! My name is Charlie and I'm a data scientist.",
    "Remember this: My birthday is April 15th.",
    "I'm working on a machine learning project about customer churn.",
    "Important: I'm allergic to peanuts.",
    "What do you remember about me?"
]

for msg in persistent_messages:
    print(f"Human: {msg}")
    response = persistent_chat.chat(msg)
    print(f"AI: {response}\n")

print("\n" + "="*50)
print(persistent_chat.get_summary())

# Clean up test file
if os.path.exists("test_memory.json"):
    os.remove("test_memory.json")

In [ ]:
# Episodic Memory - Remember specific events/episodes

class EpisodicMemory:
    """Memory system that stores and retrieves specific episodes."""
    
    def __init__(self):
        self.episodes = []  # List of episode objects
        self.llm = ChatOpenAI(temperature=0.7)
        self.current_episode = None
    
    def start_episode(self, context: str):
        """Start a new episode."""
        self.current_episode = {
            "id": len(self.episodes) + 1,
            "context": context,
            "start_time": datetime.now(),
            "events": [],
            "summary": None
        }
        return f"Started episode: {context}"
    
    def add_event(self, event: str, importance: int = 5):
        """Add an event to the current episode."""
        if self.current_episode:
            self.current_episode["events"].append({
                "event": event,
                "timestamp": datetime.now(),
                "importance": importance
            })
    
    def end_episode(self):
        """End current episode and generate summary."""
        if self.current_episode:
            # Generate episode summary
            events_str = "\n".join([e["event"] for e in self.current_episode["events"]])
            
            summary_prompt = f"""Summarize this episode in 1-2 sentences:
            Context: {self.current_episode['context']}
            Events: {events_str}
            
            Summary:"""
            
            self.current_episode["summary"] = self.llm.predict(summary_prompt)
            self.current_episode["end_time"] = datetime.now()
            
            self.episodes.append(self.current_episode)
            summary = self.current_episode["summary"]
            self.current_episode = None
            
            return f"Episode ended. Summary: {summary}"
        return "No active episode"
    
    def recall_similar_episodes(self, query: str, top_k: int = 3):
        """Find episodes similar to the query."""
        if not self.episodes:
            return []
        
        # Simple similarity based on keyword matching
        # In production, use embeddings
        query_words = set(query.lower().split())
        
        scored_episodes = []
        for episode in self.episodes:
            # Score based on context and events
            episode_text = episode["context"] + " " + " ".join([e["event"] for e in episode["events"]])
            episode_words = set(episode_text.lower().split())
            
            # Jaccard similarity
            intersection = query_words.intersection(episode_words)
            union = query_words.union(episode_words)
            score = len(intersection) / len(union) if union else 0
            
            scored_episodes.append((score, episode))
        
        # Sort by score and return top k
        scored_episodes.sort(key=lambda x: x[0], reverse=True)
        return [ep for _, ep in scored_episodes[:top_k]]
    
    def chat_with_episodic_recall(self, user_input: str) -> str:
        """Chat with episodic memory recall."""
        # Add to current episode if active
        if self.current_episode:
            self.add_event(f"User said: {user_input}")
        
        # Recall relevant episodes
        relevant_episodes = self.recall_similar_episodes(user_input, top_k=2)
        
        # Format episodes for context
        episodes_context = ""
        if relevant_episodes:
            episodes_context = "Relevant past episodes:\n"
            for ep in relevant_episodes:
                episodes_context += f"- {ep['context']}: {ep['summary']}\n"
        
        prompt = f"""You are an AI with episodic memory. You remember specific events and experiences.
        
        {episodes_context}
        
        Current episode: {self.current_episode['context'] if self.current_episode else 'No active episode'}
        
        Human: {user_input}
        
        Assistant (reference relevant episodes when appropriate):"""
        
        response = self.llm.predict(prompt)
        
        # Add response to episode
        if self.current_episode:
            self.add_event(f"AI responded: {response[:100]}...")
        
        return response

# Test episodic memory
episodic_memory = EpisodicMemory()

print("Episodic Memory System:\n")

# Episode 1: Learning Python
print(episodic_memory.start_episode("Learning Python Session"))
print()

episode1_interactions = [
    "I want to learn about Python functions.",
    "Show me an example of a decorator.",
    "How do I handle exceptions?"
]

for msg in episode1_interactions:
    print(f"Human: {msg}")
    response = episodic_memory.chat_with_episodic_recall(msg)
    print(f"AI: {response[:150]}...\n")

print(episodic_memory.end_episode())
print("\n" + "="*50 + "\n")

# Episode 2: Project Planning
print(episodic_memory.start_episode("Project Planning Meeting"))
print()

episode2_interactions = [
    "We need to build a web application.",
    "It should have user authentication.",
    "Let's use Python for the backend."
]

for msg in episode2_interactions:
    print(f"Human: {msg}")
    response = episodic_memory.chat_with_episodic_recall(msg)
    print(f"AI: {response[:150]}...\n")

print(episodic_memory.end_episode())
print("\n" + "="*50 + "\n")

# Query about past episodes
print("Querying past episodes:\n")
query = "What did we discuss about Python?"
print(f"Human: {query}")
response = episodic_memory.chat_with_episodic_recall(query)
print(f"AI: {response}")

---

## **Learner Activity 2: Build a Learning Assistant with Adaptive Memory**

Create an assistant that adapts its memory strategy based on the conversation type.

**Task**: Build an assistant that:
1. Detects conversation type (casual chat, learning session, problem-solving)
2. Uses appropriate memory strategy for each type
3. Can switch between memory modes
4. Tracks learning progress over time
5. Provides summaries and reviews

Requirements:
- Implement at least 3 different memory strategies
- Auto-detect conversation type
- Track topic progression
- Generate learning summaries
- Handle context switching

In [ ]:
# Build your adaptive learning assistant

class AdaptiveLearningAssistant:
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.7)
        # TODO: Initialize different memory systems
        # - Casual chat memory (buffer window)
        # - Learning memory (episodic + summary)
        # - Problem-solving memory (entity + vector)
        pass
    
    def detect_conversation_type(self, user_input: str) -> str:
        """Detect the type of conversation."""
        # TODO: Implement detection logic
        # Return: "casual", "learning", or "problem_solving"
        pass
    
    def switch_memory_mode(self, mode: str):
        """Switch to appropriate memory strategy."""
        # TODO: Implement mode switching
        pass
    
    def track_learning_progress(self, topic: str, understanding_level: float):
        """Track user's learning progress."""
        # TODO: Implement progress tracking
        pass
    
    def generate_learning_summary(self, topic: str) -> str:
        """Generate a summary of what was learned."""
        # TODO: Implement summary generation
        pass
    
    def chat(self, user_input: str) -> str:
        """Adaptive chat interface."""
        # TODO: Implement adaptive chat
        # - Detect conversation type
        # - Use appropriate memory
        # - Track progress if learning
        # - Generate contextual response
        pass

# TODO: Test your adaptive assistant
# Test with different conversation types
# Verify memory adaptation

# Your test code here

In [ ]:
# Solution (hidden by default)

"""
class AdaptiveLearningAssistant:
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.7)
        
        # Different memory systems
        # Casual chat - light memory
        self.casual_memory = ConversationBufferWindowMemory(
            memory_key="casual_chat",
            return_messages=True,
            k=3
        )
        
        # Learning - comprehensive memory
        self.learning_memory = ConversationSummaryMemory(
            llm=self.llm,
            memory_key="learning_summary",
            return_messages=False
        )
        
        # Problem-solving - detailed entity tracking
        self.problem_memory = ConversationEntityMemory(
            llm=self.llm,
            memory_key="problem_entities",
            return_messages=False
        )
        
        # Track learning progress
        self.learning_progress = {}
        self.conversation_history = []
        self.current_mode = None
        self.current_topic = None
        
        # Episode tracking for learning
        self.learning_episodes = []
        self.current_episode = None
    
    def detect_conversation_type(self, user_input: str) -> str:
        '''Detect the type of conversation using keywords and patterns.'''
        
        # Learning indicators
        learning_keywords = ['learn', 'teach', 'explain', 'understand', 'how does', 
                           'what is', 'why', 'tutorial', 'lesson', 'study']
        
        # Problem-solving indicators
        problem_keywords = ['solve', 'fix', 'debug', 'issue', 'problem', 'error',
                          'help me', 'stuck', 'wrong', 'broken', 'calculate']
        
        # Casual chat indicators
        casual_keywords = ['hi', 'hello', 'how are', 'weather', 'joke', 'story',
                         'opinion', 'think about', 'feeling']
        
        input_lower = user_input.lower()
        
        # Count keyword matches
        learning_score = sum(1 for kw in learning_keywords if kw in input_lower)
        problem_score = sum(1 for kw in problem_keywords if kw in input_lower)
        casual_score = sum(1 for kw in casual_keywords if kw in input_lower)
        
        # Check for questions
        if '?' in user_input:
            learning_score += 1
        
        # Determine type based on scores
        if learning_score > max(problem_score, casual_score):
            return "learning"
        elif problem_score > casual_score:
            return "problem_solving"
        else:
            return "casual"
    
    def switch_memory_mode(self, mode: str):
        '''Switch to appropriate memory strategy.'''
        if mode != self.current_mode:
            if self.current_mode == "learning" and self.current_episode:
                # End learning episode
                self._end_learning_episode()
            
            self.current_mode = mode
            
            if mode == "learning":
                # Start new learning episode
                self._start_learning_episode()
    
    def _start_learning_episode(self):
        '''Start a new learning episode.'''
        self.current_episode = {
            "start_time": datetime.now(),
            "topics": [],
            "concepts_learned": [],
            "questions_asked": [],
            "understanding_checkpoints": []
        }
    
    def _end_learning_episode(self):
        '''End current learning episode.'''
        if self.current_episode:
            self.current_episode["end_time"] = datetime.now()
            self.current_episode["duration"] = (
                self.current_episode["end_time"] - self.current_episode["start_time"]
            ).total_seconds() / 60  # in minutes
            
            # Generate episode summary
            topics = ", ".join(self.current_episode["topics"])
            self.current_episode["summary"] = f"Learned about: {topics}"
            
            self.learning_episodes.append(self.current_episode)
            self.current_episode = None
    
    def track_learning_progress(self, topic: str, understanding_level: float):
        '''Track user's learning progress on topics.'''
        if topic not in self.learning_progress:
            self.learning_progress[topic] = {
                "first_encountered": datetime.now(),
                "understanding_history": [],
                "total_interactions": 0
            }
        
        self.learning_progress[topic]["understanding_history"].append({
            "timestamp": datetime.now(),
            "level": understanding_level
        })
        self.learning_progress[topic]["total_interactions"] += 1
        
        # Update current episode
        if self.current_episode and topic not in self.current_episode["topics"]:
            self.current_episode["topics"].append(topic)
    
    def generate_learning_summary(self, topic: str = None) -> str:
        '''Generate a summary of learning progress.'''
        if topic and topic in self.learning_progress:
            progress = self.learning_progress[topic]
            latest_level = progress["understanding_history"][-1]["level"] if progress["understanding_history"] else 0
            
            return f'''Learning Summary for {topic}:
            - First encountered: {progress["first_encountered"].strftime("%Y-%m-%d")}
            - Total interactions: {progress["total_interactions"]}
            - Current understanding: {latest_level:.1%}
            - Progress trend: {'Improving' if len(progress["understanding_history"]) > 1 and 
                            progress["understanding_history"][-1]["level"] > 
                            progress["understanding_history"][0]["level"] else 'Stable'}'''
        else:
            # Overall summary
            total_topics = len(self.learning_progress)
            recent_topics = list(self.learning_progress.keys())[-3:] if self.learning_progress else []
            
            return f'''Overall Learning Summary:
            - Topics studied: {total_topics}
            - Recent topics: {', '.join(recent_topics)}
            - Learning sessions: {len(self.learning_episodes)}'''
    
    def _extract_topic(self, user_input: str) -> str:
        '''Extract the main topic from user input.'''
        # Simple extraction - in production, use NLP
        stop_words = ['the', 'a', 'an', 'is', 'are', 'what', 'how', 'why', 'can', 'you']
        words = user_input.lower().split()
        content_words = [w for w in words if w not in stop_words and len(w) > 3]
        
        if content_words:
            return content_words[0]  # Simple: take first content word
        return "general"
    
    def chat(self, user_input: str) -> str:
        '''Adaptive chat interface.'''
        
        # Detect conversation type
        conv_type = self.detect_conversation_type(user_input)
        self.switch_memory_mode(conv_type)
        
        # Extract topic for learning mode
        if conv_type == "learning":
            topic = self._extract_topic(user_input)
            self.current_topic = topic
            
            # Add to current episode
            if self.current_episode:
                self.current_episode["questions_asked"].append(user_input)
        
        # Build prompt based on mode
        if conv_type == "casual":
            memory_context = self.casual_memory.load_memory_variables({})
            prompt = f'''You are a friendly assistant having a casual conversation.
            
            Recent chat:
            {memory_context.get("casual_chat", "")}
            
            Human: {user_input}
            Assistant (be friendly and conversational):'''
            
        elif conv_type == "learning":
            memory_context = self.learning_memory.load_memory_variables({})
            progress_summary = self.generate_learning_summary(self.current_topic)
            
            prompt = f'''You are a patient teacher helping someone learn.
            
            Learning context:
            {memory_context.get("learning_summary", "")}
            
            Progress:
            {progress_summary}
            
            Human: {user_input}
            Teacher (explain clearly, use examples, check understanding):'''
            
        else:  # problem_solving
            memory_context = self.problem_memory.load_memory_variables({"input": user_input})
            
            prompt = f'''You are a problem-solving assistant.
            
            Problem context and entities:
            {memory_context.get("problem_entities", "")}
            
            Human: {user_input}
            Assistant (be systematic, break down the problem, provide solutions):'''
        
        # Get response
        response = self.llm.predict(prompt)
        
        # Save to appropriate memory
        context = {"input": user_input}
        output = {"output": response}
        
        if conv_type == "casual":
            self.casual_memory.save_context(context, output)
        elif conv_type == "learning":
            self.learning_memory.save_context(context, output)
            # Track progress (simple heuristic)
            understanding = 0.7 if "understand" in response.lower() else 0.5
            self.track_learning_progress(self.current_topic, understanding)
        else:
            self.problem_memory.save_context(context, output)
        
        # Add mode indicator to response
        mode_indicator = f"[Mode: {conv_type}] "
        
        return mode_indicator + response

# Test the adaptive assistant
print("Adaptive Learning Assistant Demo\n")
print("="*50)

assistant = AdaptiveLearningAssistant()

# Test different conversation types
test_conversations = [
    # Casual
    "Hi! How are you today?",
    # Learning
    "Can you explain what recursion is in programming?",
    "Show me an example of recursion.",
    # Problem-solving
    "I'm getting a TypeError in my Python code when I try to add a string and number.",
    # Back to learning
    "What are the main principles of object-oriented programming?",
    # Summary request
    "What have I learned so far?",
    # Casual again
    "Thanks for the help! You're really good at explaining things."
]

for msg in test_conversations:
    print(f"\nHuman: {msg}")
    response = assistant.chat(msg)
    # Show first part of response
    if len(response) > 200:
        print(f"AI: {response[:200]}...")
    else:
        print(f"AI: {response}")

print("\n" + "="*50)
print("\nLearning Progress Summary:")
print(assistant.generate_learning_summary())

if assistant.learning_progress:
    print("\nTopics covered:")
    for topic in assistant.learning_progress:
        print(f"- {topic}")
"""

print("Build an adaptive learning assistant with multiple memory strategies!")
print("The solution shows conversation type detection and appropriate memory usage.")

---

## **Instructor Activity 3: Production Memory Systems**

Let's build production-ready memory systems with databases:

In [ ]:
# Production Memory with SQLite

import sqlite3
from typing import List, Dict, Optional
import json

class SQLiteMemory:
    """Production-ready memory system using SQLite."""
    
    def __init__(self, db_path: str = "conversation_memory.db"):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self._init_database()
        self.llm = ChatOpenAI(temperature=0.7)
    
    def _init_database(self):
        """Initialize database tables."""
        cursor = self.conn.cursor()
        
        # Conversations table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS conversations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT NOT NULL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                role TEXT NOT NULL,
                content TEXT NOT NULL,
                metadata TEXT
            )
        ''')
        
        # User profiles table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS user_profiles (
                user_id TEXT PRIMARY KEY,
                profile_data TEXT NOT NULL,
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Topics table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS topics (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT NOT NULL,
                topic TEXT NOT NULL,
                importance REAL DEFAULT 0.5,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Create indexes for performance
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_session ON conversations(session_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_timestamp ON conversations(timestamp)')
        
        self.conn.commit()
    
    def save_message(self, session_id: str, role: str, content: str, metadata: Dict = None):
        """Save a message to the database."""
        cursor = self.conn.cursor()
        cursor.execute(
            'INSERT INTO conversations (session_id, role, content, metadata) VALUES (?, ?, ?, ?)',
            (session_id, role, content, json.dumps(metadata) if metadata else None)
        )
        self.conn.commit()
        return cursor.lastrowid
    
    def get_conversation_history(self, session_id: str, limit: int = 10) -> List[Dict]:
        """Retrieve conversation history for a session."""
        cursor = self.conn.cursor()
        cursor.execute(
            '''SELECT role, content, timestamp, metadata 
               FROM conversations 
               WHERE session_id = ? 
               ORDER BY timestamp DESC 
               LIMIT ?''',
            (session_id, limit)
        )
        
        rows = cursor.fetchall()
        return [
            {
                "role": row[0],
                "content": row[1],
                "timestamp": row[2],
                "metadata": json.loads(row[3]) if row[3] else None
            }
            for row in reversed(rows)  # Reverse to get chronological order
        ]
    
    def search_conversations(self, query: str, limit: int = 5) -> List[Dict]:
        """Search conversations using full-text search."""
        cursor = self.conn.cursor()
        cursor.execute(
            '''SELECT session_id, role, content, timestamp 
               FROM conversations 
               WHERE content LIKE ? 
               ORDER BY timestamp DESC 
               LIMIT ?''',
            (f'%{query}%', limit)
        )
        
        rows = cursor.fetchall()
        return [
            {
                "session_id": row[0],
                "role": row[1],
                "content": row[2],
                "timestamp": row[3]
            }
            for row in rows
        ]
    
    def update_user_profile(self, user_id: str, profile_data: Dict):
        """Update or create user profile."""
        cursor = self.conn.cursor()
        cursor.execute(
            '''INSERT INTO user_profiles (user_id, profile_data) 
               VALUES (?, ?) 
               ON CONFLICT(user_id) 
               DO UPDATE SET profile_data = ?, updated_at = CURRENT_TIMESTAMP''',
            (user_id, json.dumps(profile_data), json.dumps(profile_data))
        )
        self.conn.commit()
    
    def get_user_profile(self, user_id: str) -> Optional[Dict]:
        """Retrieve user profile."""
        cursor = self.conn.cursor()
        cursor.execute(
            'SELECT profile_data FROM user_profiles WHERE user_id = ?',
            (user_id,)
        )
        row = cursor.fetchone()
        return json.loads(row[0]) if row else None
    
    def save_topic(self, session_id: str, topic: str, importance: float = 0.5):
        """Save a topic discussed in the conversation."""
        cursor = self.conn.cursor()
        cursor.execute(
            'INSERT INTO topics (session_id, topic, importance) VALUES (?, ?, ?)',
            (session_id, topic, importance)
        )
        self.conn.commit()
    
    def get_session_summary(self, session_id: str) -> str:
        """Generate a summary of a session."""
        history = self.get_conversation_history(session_id, limit=50)
        
        if not history:
            return "No conversation history found."
        
        # Format conversation for summarization
        conversation = "\n".join([
            f"{msg['role']}: {msg['content']}" 
            for msg in history
        ])
        
        # Generate summary
        summary_prompt = f"""Summarize this conversation in 2-3 sentences:
        {conversation}
        
        Summary:"""
        
        summary = self.llm.predict(summary_prompt)
        return summary
    
    def close(self):
        """Close database connection."""
        self.conn.close()

# Test SQLite memory
print("SQLite Memory System Demo:\n")

# Create memory system
db_memory = SQLiteMemory("test_memory.db")

# Simulate conversation
session_id = "session_001"
user_id = "user_123"

# Save user profile
profile = {
    "name": "David",
    "interests": ["AI", "robotics", "space"],
    "learning_style": "visual"
}
db_memory.update_user_profile(user_id, profile)

# Simulate conversation
messages = [
    ("user", "Hi! I want to learn about neural networks."),
    ("assistant", "Great! Neural networks are computational models inspired by the human brain."),
    ("user", "How do they learn?"),
    ("assistant", "They learn through backpropagation, adjusting weights based on errors.")
]

for role, content in messages:
    db_memory.save_message(session_id, role, content)
    print(f"{role.capitalize()}: {content}")

# Save topic
db_memory.save_topic(session_id, "neural networks", importance=0.9)

print("\n" + "="*50)
print("\nRetrieved User Profile:")
retrieved_profile = db_memory.get_user_profile(user_id)
print(json.dumps(retrieved_profile, indent=2))

print("\nConversation History:")
history = db_memory.get_conversation_history(session_id)
for msg in history:
    print(f"{msg['role']}: {msg['content'][:50]}...")

print("\nSession Summary:")
summary = db_memory.get_session_summary(session_id)
print(summary)

# Clean up
db_memory.close()
import os
if os.path.exists("test_memory.db"):
    os.remove("test_memory.db")

---

## **Learner Activity 3: Build a Complete Memory Management System**

Create a comprehensive memory system for a production chatbot.

**Task**: Build a memory system that:
1. Supports multiple users with separate memory spaces
2. Implements memory compression for long conversations  
3. Provides memory search and retrieval
4. Includes memory persistence and backup
5. Handles memory privacy and data retention policies
6. Supports memory export and import

Requirements:
- Multi-user support with isolation
- Automatic memory compression
- Full-text search capability
- Data retention policies
- Export/import functionality

In [ ]:
# Build your complete memory management system

class MemoryManagementSystem:
    def __init__(self, storage_path: str = "memory_store"):
        self.storage_path = storage_path
        # TODO: Initialize storage backend
        # TODO: Set up user management
        # TODO: Configure retention policies
        pass
    
    def create_user_space(self, user_id: str) -> bool:
        """Create isolated memory space for user."""
        # TODO: Implement user space creation
        pass
    
    def compress_memory(self, user_id: str, strategy: str = "summary"):
        """Compress old memories to save space."""
        # TODO: Implement memory compression
        # Strategies: summary, selective, hierarchical
        pass
    
    def search_memory(self, user_id: str, query: str, filters: Dict = None) -> List[Dict]:
        """Search user's memory with filters."""
        # TODO: Implement semantic and keyword search
        pass
    
    def apply_retention_policy(self, user_id: str, policy: Dict):
        """Apply data retention policy."""
        # TODO: Implement retention policies
        # e.g., delete after 30 days, anonymize after 90 days
        pass
    
    def export_memory(self, user_id: str, format: str = "json") -> str:
        """Export user's memory."""
        # TODO: Implement export in different formats
        pass
    
    def import_memory(self, user_id: str, data: str, format: str = "json") -> bool:
        """Import memory data."""
        # TODO: Implement import with validation
        pass
    
    def get_memory_stats(self, user_id: str) -> Dict:
        """Get memory usage statistics."""
        # TODO: Implement statistics gathering
        pass

# TODO: Create comprehensive tests
# Test multi-user scenarios
# Test compression strategies
# Test search functionality
# Test export/import

# Your test code here

In [ ]:
# Solution (hidden by default)

"""
import os
import shutil
import hashlib
from datetime import datetime, timedelta

class MemoryManagementSystem:
    def __init__(self, storage_path: str = "memory_store"):
        self.storage_path = storage_path
        self.llm = ChatOpenAI(temperature=0.7)
        
        # Create storage directory
        os.makedirs(storage_path, exist_ok=True)
        
        # User management
        self.users = {}
        self.active_sessions = {}
        
        # Retention policies
        self.default_retention = {
            "max_age_days": 90,
            "max_messages": 10000,
            "compression_threshold": 1000,
            "anonymize_after_days": 30
        }
        
        # Initialize main database
        self.db_path = os.path.join(storage_path, "main.db")
        self._init_database()
    
    def _init_database(self):
        '''Initialize central database.'''
        import sqlite3
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Users table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                user_id TEXT PRIMARY KEY,
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                last_active DATETIME,
                memory_size INTEGER DEFAULT 0,
                message_count INTEGER DEFAULT 0,
                settings TEXT
            )
        ''')
        
        # Memory entries (centralized for search)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS memories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id TEXT NOT NULL,
                session_id TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                type TEXT,  -- 'message', 'summary', 'fact', etc.
                content TEXT,
                metadata TEXT,
                compressed BOOLEAN DEFAULT 0,
                FOREIGN KEY (user_id) REFERENCES users(user_id)
            )
        ''')
        
        # Create indexes
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_user_memories ON memories(user_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_timestamp ON memories(timestamp)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_content ON memories(content)')  # For FTS
        
        conn.commit()
        conn.close()
    
    def create_user_space(self, user_id: str) -> bool:
        '''Create isolated memory space for user.'''
        # Hash user_id for privacy
        user_hash = hashlib.sha256(user_id.encode()).hexdigest()[:16]
        user_path = os.path.join(self.storage_path, user_hash)
        
        if not os.path.exists(user_path):
            os.makedirs(user_path)
            
            # Create user-specific database
            user_db = os.path.join(user_path, "memory.db")
            conn = sqlite3.connect(user_db)
            cursor = conn.cursor()
            
            # User's conversation memory
            cursor.execute('''
                CREATE TABLE conversations (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    session_id TEXT,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    role TEXT,
                    content TEXT,
                    tokens INTEGER
                )
            ''')
            
            conn.commit()
            conn.close()
            
            # Register user in main database
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute(
                'INSERT OR IGNORE INTO users (user_id, last_active) VALUES (?, ?)',
                (user_hash, datetime.now())
            )
            conn.commit()
            conn.close()
            
            self.users[user_id] = {
                "hash": user_hash,
                "path": user_path,
                "created": datetime.now()
            }
            
            return True
        return False
    
    def compress_memory(self, user_id: str, strategy: str = "summary"):
        '''Compress old memories to save space.'''
        user_hash = hashlib.sha256(user_id.encode()).hexdigest()[:16]
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Get uncompressed memories older than 7 days
        cutoff_date = datetime.now() - timedelta(days=7)
        cursor.execute(
            '''SELECT id, content, metadata FROM memories 
               WHERE user_id = ? AND compressed = 0 AND timestamp < ?
               ORDER BY timestamp
               LIMIT 100''',
            (user_hash, cutoff_date)
        )
        
        memories_to_compress = cursor.fetchall()
        
        if not memories_to_compress:
            return "No memories to compress."
        
        if strategy == "summary":
            # Combine and summarize
            combined = "\n".join([m[1] for m in memories_to_compress[:20]])  # Take first 20
            
            summary_prompt = f'''Summarize these conversation memories concisely:
            {combined[:2000]}  # Limit input length
            
            Summary:'''
            
            summary = self.llm.predict(summary_prompt)
            
            # Save compressed summary
            cursor.execute(
                '''INSERT INTO memories (user_id, type, content, compressed) 
                   VALUES (?, 'compressed_summary', ?, 1)''',
                (user_hash, summary)
            )
            
            # Mark originals as compressed
            memory_ids = [m[0] for m in memories_to_compress[:20]]
            cursor.execute(
                f'UPDATE memories SET compressed = 1 WHERE id IN ({(",".join(["?"]*len(memory_ids)))})',
                memory_ids
            )
            
        elif strategy == "selective":
            # Keep only important messages
            important_keywords = ['important', 'remember', 'key', 'critical', 'must']
            
            for mem_id, content, metadata in memories_to_compress:
                # Check importance
                is_important = any(kw in content.lower() for kw in important_keywords)
                
                if not is_important:
                    # Mark as compressed (effectively hiding it)
                    cursor.execute(
                        'UPDATE memories SET compressed = 1 WHERE id = ?',
                        (mem_id,)
                    )
        
        elif strategy == "hierarchical":
            # Create hierarchical summaries
            # Daily -> Weekly -> Monthly
            pass  # Implementation left as exercise
        
        conn.commit()
        conn.close()
        
        return f"Compressed {len(memories_to_compress)} memories using {strategy} strategy."
    
    def search_memory(self, user_id: str, query: str, filters: Dict = None) -> List[Dict]:
        '''Search user's memory with filters.'''
        user_hash = hashlib.sha256(user_id.encode()).hexdigest()[:16]
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Build query
        sql = '''SELECT id, timestamp, type, content, metadata 
                 FROM memories 
                 WHERE user_id = ? AND compressed = 0'''
        
        params = [user_hash]
        
        # Add search condition
        if query:
            sql += ' AND content LIKE ?'
            params.append(f'%{query}%')
        
        # Add filters
        if filters:
            if 'type' in filters:
                sql += ' AND type = ?'
                params.append(filters['type'])
            
            if 'after' in filters:
                sql += ' AND timestamp > ?'
                params.append(filters['after'])
        
        sql += ' ORDER BY timestamp DESC LIMIT 20'
        
        cursor.execute(sql, params)
        results = cursor.fetchall()
        
        conn.close()
        
        return [
            {
                "id": r[0],
                "timestamp": r[1],
                "type": r[2],
                "content": r[3],
                "metadata": json.loads(r[4]) if r[4] else None
            }
            for r in results
        ]
    
    def apply_retention_policy(self, user_id: str, policy: Dict = None):
        '''Apply data retention policy.'''
        user_hash = hashlib.sha256(user_id.encode()).hexdigest()[:16]
        policy = policy or self.default_retention
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Delete old memories
        if 'max_age_days' in policy:
            cutoff = datetime.now() - timedelta(days=policy['max_age_days'])
            cursor.execute(
                'DELETE FROM memories WHERE user_id = ? AND timestamp < ?',
                (user_hash, cutoff)
            )
        
        # Limit total messages
        if 'max_messages' in policy:
            cursor.execute(
                '''DELETE FROM memories 
                   WHERE user_id = ? AND id NOT IN (
                       SELECT id FROM memories 
                       WHERE user_id = ? 
                       ORDER BY timestamp DESC 
                       LIMIT ?
                   )''',
                (user_hash, user_hash, policy['max_messages'])
            )
        
        # Anonymize old data
        if 'anonymize_after_days' in policy:
            cutoff = datetime.now() - timedelta(days=policy['anonymize_after_days'])
            # Simple anonymization - remove personal info
            cursor.execute(
                '''UPDATE memories 
                   SET content = '[ANONYMIZED]' || substr(content, -50)
                   WHERE user_id = ? AND timestamp < ? AND content NOT LIKE '[ANONYMIZED]%' ''',
                (user_hash, cutoff)
            )
        
        conn.commit()
        changes = conn.total_changes
        conn.close()
        
        return f"Retention policy applied. {changes} records affected."
    
    def export_memory(self, user_id: str, format: str = "json") -> str:
        '''Export user's memory.'''
        memories = self.search_memory(user_id, "", filters=None)
        
        if format == "json":
            return json.dumps(memories, indent=2, default=str)
        
        elif format == "csv":
            import csv
            import io
            
            output = io.StringIO()
            writer = csv.DictWriter(output, fieldnames=['timestamp', 'type', 'content'])
            writer.writeheader()
            
            for mem in memories:
                writer.writerow({
                    'timestamp': mem['timestamp'],
                    'type': mem['type'],
                    'content': mem['content']
                })
            
            return output.getvalue()
        
        elif format == "markdown":
            output = "# Memory Export\n\n"
            for mem in memories:
                output += f"## {mem['timestamp']}\n"
                output += f"Type: {mem['type']}\n\n"
                output += f"{mem['content']}\n\n"
                output += "---\n\n"
            return output
        
        return "Unsupported format"
    
    def import_memory(self, user_id: str, data: str, format: str = "json") -> bool:
        '''Import memory data.'''
        user_hash = hashlib.sha256(user_id.encode()).hexdigest()[:16]
        
        try:
            if format == "json":
                memories = json.loads(data)
                
                conn = sqlite3.connect(self.db_path)
                cursor = conn.cursor()
                
                for mem in memories:
                    cursor.execute(
                        '''INSERT INTO memories (user_id, timestamp, type, content, metadata)
                           VALUES (?, ?, ?, ?, ?)''',
                        (
                            user_hash,
                            mem.get('timestamp', datetime.now()),
                            mem.get('type', 'imported'),
                            mem.get('content', ''),
                            json.dumps(mem.get('metadata', {}))
                        )
                    )
                
                conn.commit()
                conn.close()
                return True
                
        except Exception as e:
            print(f"Import error: {e}")
            return False
    
    def get_memory_stats(self, user_id: str) -> Dict:
        '''Get memory usage statistics.'''
        user_hash = hashlib.sha256(user_id.encode()).hexdigest()[:16]
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Get counts
        cursor.execute(
            'SELECT COUNT(*), MIN(timestamp), MAX(timestamp) FROM memories WHERE user_id = ?',
            (user_hash,)
        )
        count, oldest, newest = cursor.fetchone()
        
        # Get type distribution
        cursor.execute(
            'SELECT type, COUNT(*) FROM memories WHERE user_id = ? GROUP BY type',
            (user_hash,)
        )
        type_dist = dict(cursor.fetchall())
        
        # Get compression stats
        cursor.execute(
            'SELECT compressed, COUNT(*) FROM memories WHERE user_id = ? GROUP BY compressed',
            (user_hash,)
        )
        compression_stats = dict(cursor.fetchall())
        
        conn.close()
        
        return {
            "total_memories": count,
            "oldest_memory": oldest,
            "newest_memory": newest,
            "memory_types": type_dist,
            "compressed": compression_stats.get(1, 0),
            "uncompressed": compression_stats.get(0, 0)
        }

# Test the system
print("Complete Memory Management System Demo\n")
print("="*50)

# Create system
mms = MemoryManagementSystem("test_memory_store")

# Create users
users = ["alice_123", "bob_456"]
for user in users:
    mms.create_user_space(user)
    print(f"Created memory space for {user}")

# Add some memories
import sqlite3
user_hash = hashlib.sha256("alice_123".encode()).hexdigest()[:16]

conn = sqlite3.connect(mms.db_path)
cursor = conn.cursor()

test_memories = [
    ("message", "Hello, I'm learning about AI."),
    ("message", "Can you teach me about neural networks?"),
    ("fact", "Important: My project deadline is next Friday."),
    ("message", "I understand backpropagation now!"),
]

for mem_type, content in test_memories:
    cursor.execute(
        'INSERT INTO memories (user_id, type, content) VALUES (?, ?, ?)',
        (user_hash, mem_type, content)
    )

conn.commit()
conn.close()

print("\nAdded test memories")

# Test search
print("\nSearching for 'neural':")
results = mms.search_memory("alice_123", "neural")
for r in results:
    print(f"- {r['content'][:50]}...")

# Test compression
print("\nTesting compression:")
result = mms.compress_memory("alice_123", "summary")
print(result)

# Test stats
print("\nMemory Statistics:")
stats = mms.get_memory_stats("alice_123")
for key, value in stats.items():
    print(f"- {key}: {value}")

# Test export
print("\nExporting memory:")
exported = mms.export_memory("alice_123", "json")
print(f"Exported {len(exported)} characters of data")

# Clean up
import shutil
if os.path.exists("test_memory_store"):
    shutil.rmtree("test_memory_store")
print("\nCleanup complete!")
"""

print("Build a complete production-ready memory management system!")
print("The solution includes multi-user support, compression, search, and retention policies.")

---

## **Summary and Next Steps**

Congratulations! You've mastered memory and conversation management in LangChain. You can now:

✅ Implement different types of conversation memory
✅ Build chatbots with context awareness
✅ Use entity and summary memory effectively
✅ Create episodic and semantic memory systems
✅ Build production-ready persistent memory
✅ Manage multi-user memory spaces

### **Key Takeaways:**
- **Memory Types Matter**: Choose the right memory for your use case
- **Compression is Essential**: Long conversations need smart compression
- **Persistence is Required**: Production systems need database backing
- **Privacy Matters**: Implement proper data retention and anonymization
- **Search Enables Intelligence**: Semantic search over memory improves responses

### **Next Steps:**
- **Notebook 10**: Learn about Streaming and Real-time interactions
- **Practice**: Build conversational applications with memory
- **Experiment**: Try different memory combinations
- **Scale**: Implement distributed memory systems

### **Additional Challenges:**
1. Implement graph-based memory for relationship tracking
2. Build a memory system with emotion tracking
3. Create cross-session learning that improves over time
4. Implement memory sharing between multiple agents
5. Build a privacy-preserving federated memory system